In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from tqdm import tqdm

# load ascii text and convert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X / float(n_vocab)
y = torch.tensor(dataY)

class CharModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=2, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(256, n_vocab)
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        x = self.linear(self.dropout(x))
        return x

n_epochs = 50
batch_size = 256 # 128
model = CharModel()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(X, y), shuffle=True, batch_size=batch_size)

best_model = None
best_loss = np.inf
for epoch in range(n_epochs):
    model.train()
    with tqdm(total=len(loader), ncols=80, desc=f"Epoch {epoch+1}/{n_epochs}") as pbar:
        for X_batch, y_batch in loader:
            y_pred = model(X_batch.to(device))
            loss = loss_fn(y_pred, y_batch.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pbar.set_postfix(loss=loss.item())
            pbar.update()
    # Validation
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch.to(device))
            loss += loss_fn(y_pred, y_batch.to(device))
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.4f" % (epoch, loss))

torch.save([best_model, char_to_int], "single-char.pth")

# Generation using the trained model
best_model, char_to_int = torch.load("single-char.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())
model.load_state_dict(best_model)



Total Characters:  144683
Total Vocab:  49
Total Patterns:  144583


Epoch 1/50: 100%|███████████████████| 565/565 [00:27<00:00, 20.54it/s, loss=551]


Epoch 0: Cross-entropy: 404953.3750


Epoch 2/50: 100%|███████████████████| 565/565 [00:26<00:00, 20.93it/s, loss=473]


Epoch 1: Cross-entropy: 365744.1562


...

Epoch 49/50: 100%|██████████████████| 565/565 [00:29<00:00, 19.45it/s, loss=283]


Epoch 48: Cross-entropy: 150973.6719


Epoch 50/50: 100%|██████████████████| 565/565 [00:28<00:00, 19.66it/s, loss=293]


Epoch 49: Cross-entropy: 150519.3281


<All keys matched successfully>

In [3]:
# randomly generate a prompt
filename = "wonderland.txt"
seq_length = 100
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]
pattern = [char_to_int[c] for c in prompt]

model.eval()
print('Prompt: "%s"' % prompt)
with torch.no_grad():
    for i in range(1000):
        # format input array of int into PyTorch tensor
        x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
        x = torch.tensor(x, dtype=torch.float32)
        # generate logits as output from the model
        prediction = model(x.to(device))
        # convert logits into one character
        index = int(prediction.argmax())
        result = int_to_char[index]
        print(result, end="")
        # append the new character into the prompt for the next iteration
        pattern.append(index)
        pattern = pattern[1:]
print()
print("Done.")

Prompt: "on your shoes and stockings for you now, dears? i’m
sure _i_ shan’t be able! i shall be a great deal"
 to toear as the sab. she was a little forn to the that?”

“i don’t know the way the white rabbit with the same shings and surping it to sell you think i can do br the could,

“nh, i wiol you may soe better nine a little for to thin,” she said to herself, “it would be a little wiile the borts of that is would be a little foor to the thather, and the mouse doole the had goown to her freat size. the was a little wirless all the way of sarier and sale the fiange of the court, and the shget seale in a low voice. 
“what _is_ the same thing is the semark the sea,” the mock turtle seplied in a low voice.

“what _is_ the same thing is the semark the sea,” the mock turtle seplied in a low voice.

“what _is_ the same thing is the semark the sea,” the mock turtle seplied in a low voice.

“what _is_ the same thing is the semark the sea,” the mock turtle seplied in a low voice.

“what _is_